In [1]:
import torch
from torch import nn
from torch.autograd import Variable
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from LSTM import *
from LSTMbis import *
import pandas as pd
import numpy as np
import torch.utils.data as utils
import time

import pdb

# For the notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

torch.manual_seed(1)

## TRAIN SET

In [2]:
import os
data = {}
a=0
max_len = 0
for j in range(1,4):
    for i in os.listdir('./final_dataset/train/%s'%j):
        if i.endswith('.txt'):
            a = a+1
            data['%s'%a] = pd.read_csv('./final_dataset/train/{}/{}'.format(j,i), header = None, 
                                        names = ['frameNb','id', 'x','y','Vx','Vy'],
                                           delimiter=' ')
            if len(data['%s'%a])/20>max_len:
                max_len=int(len(data['%s'%a])/20)

In [3]:
a = 0
inputs_train = np.zeros([10,len(data),4*max_len])
gt_train = np.zeros([11,len(data),4])
in_train_coord = np.zeros([10,len(data),2])
gt_train_coord = np.zeros([11,len(data),2])

for i in data:
    len_d = np.int(len(data['%s'%i])/20)
    inputs_train[:,a,:4] = data['%s'%i].loc[0:9,'x':'Vy']
    for m in range(10):
        frame = np.array(data['%s'%i].loc[m,'frameNb'])  
        for j in range(1,len_d):
            for k in range(20):
                if data['%s'%i].loc[j*20+k,'frameNb']==frame:
                    inputs_train[m,a,j*4:(j+1)*4]=data['%s'%i].loc[j*20+k,'x':'Vy']
    
    gt_train[:,a,:] = np.array(data['%s'%i].loc[9:19,['x','y','Vx','Vy']])
    in_train_coord[:,a,:] = np.array(data['%s'%i].loc[0:9,['x','y']])
    gt_train_coord[:,a,:] = np.array(data['%s'%i].loc[9:19,['x','y']])
    a +=1


inputs_train = torch.from_numpy(inputs_train).float()
gt_train = torch.from_numpy(gt_train).float()
inputs_train = inputs_train.permute([1,0,2])
gt_train = gt_train.permute([1,0,2])


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/rodolphefarrando/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-17c46f7a4be5>", line 14, in <module>
    if data['%s'%i].loc[j*20+k,'frameNb']==frame:
  File "/Users/rodolphefarrando/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py", line 1363, in __getitem__
    if self._is_scalar_access(key):
  File "/Users/rodolphefarrando/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py", line 1537, in _is_scalar_access
    if not ax.is_unique:
  File "/Users/rodolphefarrando/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/range.py", line 228, in is_unique
    @property
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/rodolphefarrando/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell

KeyboardInterrupt: 

In [ ]:
inputs_train.shape, gt_train.shape

In [ ]:
from torch import optim

lr = 0.01
lstm = LSTM(4*max_len,4)
optimizer = optim.SGD(lstm.parameters(), lr=lr)
criterion = nn.MSELoss()

In [ ]:
traindataset = utils.TensorDataset(inputs_train, gt_train[:,1:,:])
trainloader = utils.DataLoader(traindataset, batch_size=2, shuffle=True)

epochs = 10
steps = 0
print_every = 200
running_loss = 0 #### DOD

for e in range(epochs):
    start = time.time()
    if (e+1)%5==0:
        lr /= 5
        optimizer = optim.SGD(lstm.parameters(), lr=0.005)

    for train_coord, ground_tru in iter(trainloader):
        steps += 1
        
        train_coord = train_coord.permute([1,0,2])
        ground_tru = ground_tru.permute([1,0,2])

        in_train = Variable(train_coord)
        targets = Variable(ground_tru)
        optimizer.zero_grad()
        #print(in_train.shape)
        #print(targets.shape)
        out = lstm.forward(in_train)
        out_bis = out[:,:,0:2].clone()
        #print(out_bis.shape)    
        for i in range(10):
            if i == 0:
                out_bis[i, :, 0:2] = in_train[-1, :, 0:2] + out[i, :, 2:]*0.4
            else:
                out_bis[i, :, 0:2] = out[i - 1, :, 0:2] + out[i, :, 2:]*0.4
        #pdb.set_trace()
        loss = (criterion(out, targets) + criterion(out_bis[1:-1,:,:], targets[1:-1,:,0:2])
                                + 2*criterion(out_bis[-1,:,:], targets[-1,:,0:2]) 
                                + 2*criterion(out_bis[0,:,:], targets[0,:,0:2]))
        loss.backward()
        optimizer.step()
        
        running_loss += loss.data[0]

    
        if steps % print_every == 0:
                stop = time.time()
                print("Epoch: {}/{}..".format(e+1, epochs),
                  "Loss: {:.4f}..".format(running_loss/print_every),
                  "{:.4f} s/batch".format((stop - start)/print_every),
                  "{:.8f} lr".format(lr)
                 )
                running_loss = 0
                start = time.time()

In [ ]:
inputs_pred = inputs_train.permute([1,0,2])
output_final = lstm.predict(inputs_pred)
output_final = output_final.data.numpy()

## Post processing step
Go back to coordinate:
We have Vx and Vy and we want x and y.
$ V = d/t$
$ d = V*t$
Here t = 0.4s between each point.
Start from data at index 9. Then we add d_x and d_y to the last x and y.

In [ ]:
output_final.shape

In [ ]:
output_coord=np.zeros([11,len(data),2])
for j in range(11):
    for i in range(len(data)):
        if j==0:
            output_coord[j,i,0:2] = in_train_coord[9,i,0:2]
        else:
            output_coord[j,i,0:2] = output_final[j-1,i,0:2]+output_final[j-1,i,2:]*0.4
        

In [ ]:
for i in range(20):
    plt.figure(figsize=(12, 7))

    plt.plot(in_train_coord[:,(i*20)+3000,0],in_train_coord[:,i*20+3000,1],c='b')
    plt.plot(gt_train_coord[:,i*20+3000,0],gt_train_coord[:,i*20+3000,1],c='k')
    plt.plot(output_coord[:,i*20+3000,0],output_coord[:,i*20+3000,1],c='r')
    plt.axis('equal')